In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 50.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=eea015327d5e38cf5d149fde9d8fb435b0bb1e49b12d1b2fb92b700a93c5ce72
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
if __name__ == "__main__":
  conf = SparkConf().setAppName("MyCode").setMaster("local[2]")
  sc = SparkContext(conf = conf)

In [4]:
data = sc.textFile("BigMart_Sales.csv")

In [8]:
data.take(4)

['Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales',
 'FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,4235.475',
 'DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,604.4969',
 'FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2358.5261']

In [9]:
data.getNumPartitions()

2

In [10]:
data.count()

8524

# Remove Heading

In [11]:
header = data.first()
header

'Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales'

In [12]:
newdata = data.filter(lambda x: x!=header)
newdata.take(5)

['FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,4235.475',
 'DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,604.4969',
 'FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2358.5261',
 'FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,Medium,Tier 3,Grocery Store,811.6203',
 'NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,736.0226']

# First Function




### Total Item_Outlet_Sales of given Outlet_Type having given item_fat as per item_type

In [13]:
import pandas as pd
class function1:
  def __init__(self, marttype, fat):
    self.marttype = marttype
    self.fat = fat

  def filters(self, x):
    a = x.split(",")[10]
    b = x.split(",")[2]
    if a == self.marttype and b == self.fat:
      return (x.split(",")[4], x.split(",")[11])
  
  def sale(self, x, y):
    return float(x) + float(y)
  
  def sort(self, x):
    a = float(x[1])
    return a

  def table(self,foodrdd):
    dic = {}
    food_name = []
    total_sale = []
    for i in foodrdd.collect():
      food_name.append(i[0])
      total_sale.append(i[1])
    dic["Food_name"] = food_name
    dic["Total_sale"] = total_sale
    return pd.DataFrame(dic)

if __name__ == "__main__":
  marttype = input("1. Outlet Type :- \n 1.Supermarket Type1\n 2.Supermarket Type2\n 3.Supermarket Type3\n 4.Grocery Store\nEnter Outlet Type: ")
  fat = input("2. Item Fat:- \n 1.Low Fat\n 2.Regular\nEnter Item Fat Content: ")
  fun1 = function1(marttype, fat)
  newrdd = newdata.filter(fun1.filters).map(fun1.filters).reduceByKey(fun1.sale).sortBy(fun1.sort)
  result1 = fun1.table(newrdd)

1. Outlet Type :- 
 1.Supermarket Type1
 2.Supermarket Type2
 3.Supermarket Type3
 4.Grocery Store
Enter Outlet Type: Supermarket Type1
2. Item Fat:- 
 1.Low Fat
 2.Regular
Enter Item Fat Content: Low Fat


In [14]:
result1

,Food_name,Total_sale
0,Seafood,4.248980e+04
1,Breakfast,6.637590e+04
2,Starchy Foods,1.441969e+05
3,Meat,1.863575e+05
4,Breads,1.926356e+05
5,Others,2.194258e+05
6,Hard Drinks,3.033961e+05
7,Baking Goods,3.767422e+05
8,Canned,4.824855e+05
9,Soft Drinks,5.065147e+05


# Second Function

### Total Sales of give Outlet_Location of given Outlet_Size

In [16]:
class function2:
  def __init__(self, location, size):
    self.location = location
    self.size = size
  
  def myfilter(self,x):
    a = x.split(",")[9]
    b = x.split(",")[8]
    if a == self.location and self.size:
      return x.split(",")[11]
  
  def total(self,x,y):
    return float(x) + float(y)

if __name__ == "__main__":
  location = input("1. Outlet Location:- \n 1.Tier 1\n 2.Tier 2\n 3.Tier 3\nEnter Outlet Location: ")
  size = input("2. Outlet Size:- \n 1.Medium\n 2.Small\n 3.High\nEnter Outlet Size: ")
  fun2 = function2(location, size)
  newrdd2 = newdata.filter(fun2.myfilter).map(fun2.myfilter).reduce(fun2.total)
  print("Total Sales Of Outlet_Location '{}' of Outlet_Size '{}' is ${}".format(location, size, newrdd2))


1. Outlet Location:- 
 1.Tier 1
 2.Tier 2
 3.Tier 3
Enter Outlet Location: Tier 1
2. Outlet Size:- 
 1.Medium
 2.Small
 3.High
Enter Outlet Size: Medium
Total Sales Of Outlet_Location 'Tier 1' of Outlet_Size 'Medium' is $4480518.115138998


# Third Function

### According to item fat and sales greater then 4000 filter all item type, item price, item weight

In [44]:
import pandas as pd
class function3:
  def __init__(self, fat):
    self.fat = fat
  
  def myfilter(self,x):
    a = x.split(",")
    b = x.split(",")
    if a[2] == self.fat and float(b[11]) >= 4000:
      return (x.split(",")[4], x.split(",")[5])
    
  def sort(self,x):
    a = float(x[1])
    return a
  
  def table(self, data):
    dic = {}
    food = []
    price = []
    for i in data.collect():
      food.append(i[0])
      price.append(i[1])
    dic["Food_Name"] = food
    dic["Price"] = price
    return pd.DataFrame(dic)
  
  def group_food(self, x,y):
    return x

if __name__ == "__main__":
  fat = input("2. Item Fat:- \n 1.Low Fat\n 2.Regular\nEnter Item Fat Content: ")
  fun3 = function3(fat)
  newrdd3 = newdata.filter(fun3.myfilter).map(fun3.myfilter).sortBy(fun3.sort).reduceByKey(fun3.group_food)
  result3 = fun3.table(newrdd3)

2. Item Fat:- 
 1.Low Fat
 2.Regular
Enter Item Fat Content: Low Fat


In [45]:
result3

,Food_Name,Price
0,Baking Goods,127.5362
1,Health and Hygiene,139.9154
2,Snack Foods,140.9154
3,Others,143.847
4,Breads,162.6552
5,Hard Drinks,171.7422
6,Household,125.9336
7,Fruits and Vegetables,140.6154
8,Meat,143.7812
9,Frozen Foods,147.5392


# Fourth Function

### Year wise Total Sales

In [52]:
import pandas as pd
class function4:
  def mymap(x):
    return (x.split(",")[7], x.split(",")[11])

  def myreduce(x,y):
    a = float(x) + float(y)
    return a
  
  def sort(x):
    a = x[0]
    return a

  def table(data):
    dic = {}
    year = []
    sales = []
    for i in data.collect():
      year.append(i[0])
      sales.append(i[1])
    dic["Year"] = year
    dic["Total_Sales"] = sales
    return pd.DataFrame(dic)

fun4 = function4
newrdd4 = newdata.map(fun4.mymap).reduceByKey(fun4.myreduce).sortBy(fun4.sort)
result4 = fun4.table(newrdd4)


In [53]:
result4

,Year,Total_Sales
0,1985,3.634365e+06
1,1987,2.140092e+06
2,1997,2.118801e+06
3,1998,1.909775e+05
4,1999,2.182020e+06
5,2002,2.040726e+06
6,2004,2.262986e+06
7,2007,2.165617e+06
8,2009,1.855210e+06


# Fifth Function

### Total Sale of particular item having particular fat

In [91]:
class function5:
  def __init__(self, food, fat):
    self.food = food
    self.fat = fat
  
  def myfilter(self,x):
    if self.food == x.split(",")[4] and self.fat == x.split(",")[2]:
      return x.split(",")[11]
  
  def total(self,x,y):
    result = float(x) + float(y)
    return result
  
if __name__ == "__main__":
  fat = input("1. Item Fat:- \n 1.Regular \n 2.Low Fat \nEnter Item Fat: ")
  food = input("2. Item Name:- \nEnter Outlet Size: ")
  fun5 = function5(food, fat)
  newrdd5 = newdata.filter(fun5.myfilter).map(fun5.myfilter).reduce(fun5.total)
  result5 = "Total Sales Of '{}' having '{}' fat is ${}".format(food, fat, newrdd5)


1. Item Fat:- 
 1.Regular 
 2.Low Fat 
Enter Item Fat: Regular
2. Item Name:- 
Enter Outlet Size: Soft Drinks


In [92]:
result5

"Total Sales Of 'Soft Drinks' having 'Regular' fat is $129255.886018"